In [1]:
#运行时间
import datetime
now = datetime.datetime.now() # 返回示例：2020-12-22 17:55:14.282811「包含微秒」
now.strftime("%Y-%m-%d %H:%M:%S") # 返回示例：2020-12-22 17:55:14
start = datetime.datetime.now()
print(start)

#导入原始数据
# encoding=utf-8
import pandas as pd
from sklearn.metrics import precision_score
data = pd.read_excel('../竞赛用数据集和说明_1021.xlsx',sheet_name='训练')
#data = pd.read_excel('data/test.xlsx',sheet_name=0)
data = data.drop(['pax_name','emd_lable','pax_passport','seg_dep_time','flt_cnt_y2', 'flt_cnt_y3','birth_date','recent_flt_day'],axis=1)
data = data.fillna(0)

2021-03-20 15:20:31.677908


KeyboardInterrupt: 

In [ ]:
#列加和
lie_sum = []
for i in data.columns.values.tolist():
    try:
        lie_sum.append(len(data[data[i]==0]))
    except Exception:
        lie_sum.append('ss')

In [ ]:
#去除缺失过多数据
kk = []
for i in range(len(lie_sum)):
    if(lie_sum[i]>22000):
        kk.append(data.columns[i])
data1 = data.drop(kk,axis=1)
print(data.shape)

In [ ]:
#训练集划分
from sklearn.model_selection import train_test_split
data.fillna(0)

y = data['emd_lable2']

X = data1.drop(['emd_lable2'],axis=1)

In [ ]:
#特征向量化
from sklearn.feature_extraction import  DictVectorizer
#sparse=False意思是不产生稀疏矩阵
vec = DictVectorizer(sparse=False)

X = vec.fit_transform(X.to_dict(orient='records'))

In [ ]:
#特征降维
from  sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(0.95*(1-0.95))
X1 = sel.fit_transform(X)

from sklearn.decomposition import PCA
pca = PCA(n_components=20)   #降到2维
pca.fit(X)                  #训练
newX=pca.fit_transform(X1)   #降维后的数据

In [ ]:
print(newX.shape)
print(X1.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=5)

In [ ]:
#伯努利朴素贝叶斯
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

gnb = BernoulliNB()
clf = gnb.fit(X_train, y_train)
y_pred = clf.predict(X_test)

#计算精度
print (clf.score(X_train, y_train))
print ('训练集准确率：', accuracy_score(y_train, clf.predict(X_train)))
print ('训练集精度：', precision_score(y_train, clf.predict(X_train)))
print ('训练集召回率：', recall_score(y_train, clf.predict(X_train)))
print (clf.score(X_test, y_test))
print ('测试集准确率：', accuracy_score(y_test, clf.predict(X_test)))
print ('测试集精度：', precision_score(y_test, clf.predict(X_test)))
print ('测试集召回率：', recall_score(y_test, clf.predict(X_test)))

In [ ]:
#高斯朴素贝叶斯
from sklearn.naive_bayes import GaussianNB

gs = GaussianNB()
clf = gs.fit(X_train, y_train)
y_pred = clf.predict(X_test)

#计算精度
print (clf.score(X_train, y_train))
print ('训练集准确率：', accuracy_score(y_train, clf.predict(X_train)))
print ('训练集精度：', precision_score(y_train, clf.predict(X_train)))
print ('训练集召回率：', recall_score(y_train, clf.predict(X_train)))
print (clf.score(X_test, y_test))
print ('测试集准确率：', accuracy_score(y_test, clf.predict(X_test)))
print ('测试集精度：', precision_score(y_test, clf.predict(X_test)))
print ('测试集召回率：', recall_score(y_test, clf.predict(X_test)))

In [ ]:
#决策树
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

#clf = DecisionTreeClassifier(min_samples_split=15,random_state=12,criterion='entropy',max_depth=15) # 初始化
clf = DecisionTreeClassifier(min_samples_split=3,criterion='entropy',random_state=12) # 初始化
clf = clf.fit(X_train,y_train) # 拟合
#计算精度
print (clf.score(X_train, y_train))
print ('训练集准确率：', accuracy_score(y_train, clf.predict(X_train)))
print ('训练集精度：', precision_score(y_train, clf.predict(X_train)))
print ('训练集召回率：', recall_score(y_train, clf.predict(X_train)))
print (clf.score(X_test, y_test))
print ('测试集准确率：', accuracy_score(y_test, clf.predict(X_test)))
print ('测试集精度：', precision_score(y_test, clf.predict(X_test)))
print ('测试集召回率：', recall_score(y_test, clf.predict(X_test)))

In [ ]:
#神经网络
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(200), activation="relu",
                 solver='adam', alpha=0.0001,
                 batch_size='auto', learning_rate="constant",
                 learning_rate_init=0.001, power_t=0.5, max_iter=200 ,tol=1e-4)
clf = clf.fit(X_train, y_train)

#计算精度
print (clf.score(X_train, y_train))
print ('训练集准确率：', accuracy_score(y_train, clf.predict(X_train)))
print ('训练集精度：', precision_score(y_train, clf.predict(X_train)))
print ('训练集召回率：', recall_score(y_train, clf.predict(X_train)))
print (clf.score(X_test, y_test))
print ('测试集准确率：', accuracy_score(y_test, clf.predict(X_test)))
print ('测试集精度：', precision_score(y_test, clf.predict(X_test)))
print ('测试集召回率：', recall_score(y_test, clf.predict(X_test)))
print ('测试集召回率：', recall_score(y_test, clf.predict(X_test)))
